In [1]:
from imutils.perspective import four_point_transform

from imutils import contours
import numpy as np

import imutils
import cv2
import random


In [101]:
image = cv2.imread("testcase2.jpg")
print('Original Dimensions : ',image.shape)
scale_percent = 40 # percent of original size
width = int(image.shape[1] * scale_percent / 100)
height = int(image.shape[0] * scale_percent / 100)
dim = (width, height) 
# resize image
resized = cv2.resize(image, dim, interpolation = cv2.INTER_AREA)
 
gray = cv2.cvtColor(resized, cv2.COLOR_BGR2GRAY)



blurred = cv2.GaussianBlur(gray, (5, 5),1.44)


#edged = cv2.Canny(blurred, 75, 200)

ELLIPSE_KERNEL = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (70, 70))
Black_Hat = cv2.morphologyEx(blurred, cv2.MORPH_BLACKHAT, ELLIPSE_KERNEL)
cv2.imshow("Black_Hat", Black_Hat)
cv2.waitKey(0)
cv2.destroyAllWindows()

edged = cv2.Canny(Black_Hat, 245, 250)
#thresh = cv2.threshold(Black_Hat.copy(), 0, 255,cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)[1]
#cv2.imshow("thresh", thresh)
#cv2.waitKey(0)
#cv2.destroyAllWindows()





#contours = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL,
#cv2.CHAIN_APPROX_SIMPLE)
#contours = imutils.grab_contours(contours)

contours, _ = cv2.findContours(edged.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
hierarchy = _[0]
All_cnts= cv2.drawContours(resized.copy(), contours, -1, (0, 0, 255), 2)
print("sum of contour:",len(contours))
# show the output image
cv2.imshow("All Contours", All_cnts)
cv2.waitKey(0)
clone = resized.copy()
questionCnts = []
i = 0
for c in contours:
    approx = cv2.approxPolyDP(c, 0.01* cv2.arcLength(c,True), True)
    area = cv2.contourArea(c)
    
    if ((len(approx) > 7) & (area > 500) ):
        i +=1
        questionCnts.append(c)
        #questionCnts = sorted(questionCnts, key=lambda x: cv2.contourArea(x),reverse=False)

print("num of contour",i)
cv2.drawContours(clone, questionCnts,  -1, (255,0,0), 2)
cv2.imshow('khoanh contour',clone)
cv2.waitKey(0)

#contourSort = sort_contours(questionCnts, method="top-to-bottom")[0]
#boundingBoxes = [cv2.boundingRect(c) for c in questionCnts]
#(contourSort, boundingBoxes) = zip(*sorted(zip(questionCnts, boundingBoxes),key=lambda b:b[1][0], reverse=True))
#contourSort = sorted(contourSort, key=lambda x: cv2.contourArea(x),reverse=False)
#boundingBoxes1 = [cv2.boundingRect(c) for c in contourSort]
#(contourSort1, boundingBoxes1) = zip(*sorted(zip(questionCnts, boundingBoxes),key=lambda b:b[1][1], reverse=False))
#print(len(contourSort))



    

Original Dimensions :  (1057, 3001, 3)
sum of contour: 253
num of contour 176


-1

In [102]:

thresh_img = cv2.adaptiveThreshold(blurred, 255,cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY_INV, 45, 15)

cv2.imshow("Thresh", thresh_img)
# perform connected components analysis on the thresholded images and
# initialize the mask to hold only the "large" components we are interested in
_, labels = cv2.connectedComponents(thresh_img)
mask = np.zeros(thresh_img.shape, dtype="uint8")

# loop over the unique components
for (i, label) in enumerate(np.unique(labels)):
# if this is the background label, ignore it

    labelMask = np.zeros(thresh_img.shape, dtype="uint8")
    labelMask[labels == label] = 255
    numofPixels = cv2.countNonZero(labelMask)
   # print(" numPixels:".format(numofPixels))
    if numofPixels > 500 and numofPixels < 2000:
        mask = cv2.add(mask, labelMask)

cv2.imshow("Mask", mask)
cv2.waitKey(0)

# find external contours in the image
cnts, _ = cv2.findContours(mask.copy(), cv2.RETR_EXTERNAL, 
cv2.CHAIN_APPROX_SIMPLE)
clone = resized.copy()

# loop over the contours
AnswersCnt = []
i = 0
for c in cnts:

    (x, y, w, h) = cv2.boundingRect(c)

    ar = w/float(h)

    if w >= 20 and h >= 20 and ar >= 0.1 and ar <= 1.2:
        i+=1
        AnswersCnt.append(c)
# show the output image
print("so luong contour",i)
cv2.drawContours(clone, AnswersCnt,  -1, (0,255,0), 2)
cv2.imshow('khoanh contour',clone)
cv2.waitKey(0)
#(cnts, boundingBoxes) = contours.sort_contours(AnswersCnt, method="top-to-bottom")



so luong contour 16


-1

In [103]:
def sort_contours(cnts, method="left-to-right"):
    # initialize the reverse flag and sort index
    reverse = False
    i = 0
    # handle if we need to sort in reverse
    if method == "right-to-left" or method == "bottom-to-top":
        reverse = True
    # handle if we are sorting against the y-coordinate rather than
    # the x-coordinate of the bounding box
    if method == "top-to-bottom" or method == "bottom-to-top":
        i = 1
    # construct the list of bounding boxes and sort them from top to
    # bottom
    boundingBoxes = [cv2.boundingRect(c) for c in cnts]
    (cnts, boundingBoxes) = zip(*sorted(zip(cnts, boundingBoxes),
        key=lambda b:b[1][i], reverse=reverse))
    # return the list of sorted contours and bounding boxes
    return (cnts, boundingBoxes)

In [106]:
anscnts = []
ansfi =resized.copy() 
tempSortContour = sort_contours(questionCnts, method="top-to-bottom")[0]
for (q, i) in enumerate(np.arange(0, len(tempSortContour), 22)):
    
    # gán màu cho contour
    color = (random.randint(0, 255),random.randint(0, 255),random.randint(0, 255))
    # Sap xep cac contour theo cot
    cnts = sort_contours(tempSortContour[i:i + 22])[0]
    anscnts.append(cnts)

# loop over the contours


for i in range(len(anscnts)):
    for j in range( len(anscnts[i])):
        M = cv2.moments(anscnts[i][j])
        cX = int(M["m10"] / M["m00"])
        cY = int(M["m01"] / M["m00"])
        
        #print(cX)
        for each in AnswersCnt:
            N = cv2.moments(each)
            nX = int(N["m10"] / N["m00"])
            nY = int(N["m01"] / N["m00"])
            dist = cv2.pointPolygonTest(anscnts[i][j],(nX,nY),True)
            if dist >= 0:
                #print(j)
                if j == 0: #vang
                    cv2.putText(clone, "Vang", (cX+1030, cY), cv2.FONT_HERSHEY_SIMPLEX,1.0, (0, 0, 255), 2)
                    
                if(j <= 11 and j > 0):
                    j = j-1
                    cv2.putText(clone, str(j), (1100, nY), cv2.FONT_HERSHEY_SIMPLEX,1.0, (0, 0, 255), 2)
                    
                if(j > 11):
                    j = j -12
                    cv2.putText(clone, "."+str(j), (1125, nY), cv2.FONT_HERSHEY_SIMPLEX,1.0, (0, 0, 255), 2)
                
cv2.imshow("Final Answer",clone)
cv2.waitKey(0)
               
       




-1